In [1]:
# %%
from numpy.core.fromnumeric import var
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize, Lambda, Resize, ConvertImageDtype, Grayscale
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.trainer import Trainer
import matplotlib.pyplot as plt
import torchvision

torch.cuda.is_available()

/home/bora/miniconda3/envs/vae-inn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [ ]:

pl.seed_everything(42)

# %%
#add tensorboard logger
logger = TensorBoardLogger("tb_logs_vanilla", name="my_model")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = Compose(
    [ToTensor()])
train_set = datasets.MNIST(root='data', train=True,
                              download=True, transform=transform)

# %%
from model import ConvInnModel

model = ConvInnModel()
BATCH_SIZE = 32
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
y, _, _ = model(torch.randn((1,1,28,28)))
y.shape

# %%
if DEVICE == 'cpu':
  devices = 0
else:
  devices = 1

class ModelSaveCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is starting")

    def on_train_epoch_start(self, trainer: Trainer, pl_module: pl.LightningModule) -> None:
        if trainer.current_epoch % 2 == 0:
          torch.save(model.state_dict(), "model_vanilla_mnist.pt")
          BATCH_SIZE = 20
          samples = model.inn.inverse(torch.randn((BATCH_SIZE, 1, 28, 28))).detach()
          grid_img = torchvision.utils.make_grid(samples, nrow=5)
          torchvision.utils.save_image(grid_img, f"outputs_mnist/epoch_{trainer.current_epoch}.png")

    def on_train_end(self, trainer, pl_module):
        print("Training is ending")



torch.cuda.is_available()

In [ ]:
# trainer = pl.Trainer(max_epochs=10, devices=devices,logger=logger,callbacks=[checkpoint_callback])  # Set devices=1 for GPU training
trainer = pl.Trainer(max_epochs=200, devices=1, logger=logger, callbacks=[ModelSaveCallback()])  # Set devices=1 for GPU training

# Start training
trainer.fit(model, train_dataloaders=train_loader)

# Save the model for later inference.
torch.save(model.state_dict(), "model_vanilla.pt")